In [1]:
import os

if not os.getcwd().endswith('Dissertation'):
    os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from src.ModelUsage import call_model

year = 2019

with open(f'Data\\test_csv\\{year}\\train_to.txt', 'r') as f:
    train_to = {i.rstrip().split(' - ')[0]:
                      {
                          'Q1': int(i.rstrip().split(' - ')[1]),
                          'Q2': int(i.rstrip().split(' - ')[2]),
                          'Tyres': i.rstrip().split(' - ')[3]
                       }
                   for i in f.readlines()}
train_to

{'03_17': {'Q1': 26, 'Q2': 54, 'Tyres': 'Soft'},
 '03_31': {'Q1': 106, 'Q2': 133, 'Tyres': 'Soft'},
 '04_14': {'Q1': 180, 'Q2': 207, 'Tyres': 'Soft'},
 '04_28': {'Q1': 277, 'Q2': 295, 'Tyres': 'Soft'},
 '05_26': {'Q1': 373, 'Q2': 423, 'Tyres': 'Soft'},
 '06_09': {'Q1': 428, 'Q2': 457, 'Tyres': 'Soft'},
 '06_30': {'Q1': 449, 'Q2': 479, 'Tyres': 'Soft'},
 '07_28': {'Q1': 466, 'Q2': 495, 'Tyres': 'Soft'},
 '09_22': {'Q1': 455, 'Q2': 481, 'Tyres': 'Soft'},
 '09_29': {'Q1': 399, 'Q2': 427, 'Tyres': 'Soft'},
 '10_13': {'Q1': 377, 'Q2': 406, 'Tyres': 'Soft'},
 '11_03': {'Q1': 358, 'Q2': 385, 'Tyres': 'Soft'},
 '12_01': {'Q1': 356, 'Q2': 384, 'Tyres': 'Soft'}}

In [3]:
def run_model(event_date, qual_session, loops: int=10):
    csv_location = f'Data\\test_csv\\{year}\\{event_date}.csv'

    cutoff = 16 if qual_session == 'Q1' else 11



    data = pd.read_csv(csv_location).copy()

    session_data = data[(data['Date'] == event_date) & (data['SessionName'] == qual_session)]

    minimum_laps = session_data.groupby('DriverName')['LapTime'].min()
    cutoff = (np.sort(minimum_laps.to_numpy())[cutoff - 1])

    means = []
    stds = []
    confs = []
    losses = []
    for i in range(loops):
        _, (mean, std, con), loss = call_model(csv_location, qual_session, event_date, train_to=train_to[event_date][qual_session], return_std=True, return_con=True, return_loss = True, pbar=False, tyres=train_to[event_date]['Tyres'])
        means.append(mean)
        stds.append(std)
        confs.append(con)
        losses.append(loss)
    return means, stds, confs, cutoff, losses

results = dict()

for date in tqdm(list(train_to.keys())):
    if os.path.exists(f'Data\\test_csv\\{year}_results\\{date}.csv'):
        results[date] = pd.read_csv(f'Data\\test_csv\\{year}_results\\{date}.csv').to_dict()

        for key, value in results[date].items():
            results[date][key] = [i for i in value.values()]
    else:
        results[date] = dict()
        for key in ['Q1_means', 'Q1_stds', 'Q1_confs', 'Q1_cutoff', 'Q2_means', 'Q2_stds', 'Q2_confs', 'Q2_cutoff']:
            results[date][key] = []

  0%|          | 0/13 [00:00<?, ?it/s]

In [4]:
losses = {
    'Q1': [],
    'Q2': []
}

for date in tqdm(list(train_to.keys())):
    for session in ['Q1', 'Q2']:
        print(date, session)
        means, stds, confs, cutoff, loss = run_model(date, session, loops=39 )
        results[date][f'{session}_means'] += means
        results[date][f'{session}_stds'] += stds
        results[date][f'{session}_confs'] += confs
        results[date][f'{session}_cutoff'] = cutoff
        losses[session] += loss

  0%|          | 0/13 [00:00<?, ?it/s]

03_17 Q1
03_17 Q2
03_31 Q1
03_31 Q2
04_14 Q1
04_14 Q2
04_28 Q1
04_28 Q2
05_26 Q1
05_26 Q2
06_09 Q1
06_09 Q2
06_30 Q1
06_30 Q2
07_28 Q1
07_28 Q2
09_22 Q1
09_22 Q2
09_29 Q1
09_29 Q2
10_13 Q1
10_13 Q2
11_03 Q1
11_03 Q2
12_01 Q1
12_01 Q2


In [5]:
for qual in ['Q1', 'Q2']:
    means = np.concatenate([results[date][f'{qual}_means'] - results[date][f'{qual}_cutoff'] for date in results])
    print(np.mean(np.abs(means)))

    stds = np.concatenate([results[date][f'{qual}_stds'] for date in results])
    print(np.mean(np.abs(stds)))

0.4513057114099714
0.6286413343575775
0.19691973931789836
0.2008442640933108


In [6]:
for key, value in results.items():
    dataframe = pd.DataFrame.from_dict(value)
    dataframe.to_csv(f'Data\\test_csv\\{year}_results\\{key}.csv', index=False)

In [7]:
for key in losses:
    print(np.mean(losses[key]))

0.7636472055499741
0.4883881448382998


In [8]:
stds = []
print(results.keys())
for key, value in results.items():
    stds += (results[key]['Q2_stds'])
print(np.mean(stds))
print(1.96 * np.std(stds))

dict_keys(['03_17', '03_31', '04_14', '04_28', '05_26', '06_09', '06_30', '07_28', '09_22', '09_29', '10_13', '11_03', '12_01'])
0.2008442640933108
0.2043054620211786
